In [1]:
%pylab inline

import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy import integrate
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')
from pylab import *
import shutil

import parameters as pam
import lattice as lat
import variational_space as vs
import hamiltonian as ham
import basis_change as basis
import get_state as getstate
import utility as util
import plotfig as fig
import ground_state as gs
import lanczos
import time
start_time = time.time()
M_PI = math.pi
                  
#####################################
def compute_Aw_main(ANi,ACu,epCu,epNi,epbilayer,tpd,tpp,tpzd,tz_a1a1,tz_b1b1,pds,pdp,pps,ppp,Upp,\
                    d_Ni_double,d_Cu_double,p_double,pz_double,double_Ni_part,hole345_Ni_part, double_Cu_part,\
                    hole345_Cu_part, idx_Ni,idx_Cu, U_Ni, \
                    S_Ni_val, Sz_Ni_val, AorB_Ni_sym, \
                    U_Cu, S_Cu_val, Sz_Cu_val, AorB_Cu_sym):  
    if Norb==8 or Norb==5:
        fname = 'epCu'+str(epCu)+'epNi'+str(epNi)+'_tpd'+str(tpd)+'_tpp'+str(tpp) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta) +'_ANi'+str(ANi) \
                  + '_ACu'+str(ACu) + '_B'+str(B) + '_C'+str(C) +'_tz_a1a1' +str(tz_a1a1)  +'_tz_b1b1' +str(tz_b1b1)                 
        flowpeak = 'Norb'+str(Norb)+'_tpp'+str(tpp)+'_Mc'+str(Mc)+'_eta'+str(eta)
    elif Norb==10 or Norb==11 or Norb==12:
        fname = 'epCu'+str(epCu)+'epNi'+str(epNi)+'_pdp'+str(pdp)+'_pps'+str(pps)+'_ppp'+str(ppp) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta) \
                  +'_ANi'+str(ANi) + '_ACu'+str(ACu) + '_B'+str(B) + '_C'+str(C)
        flowpeak = 'Norb'+str(Norb)+'_pps'+str(pps)+'_ppp'+str(ppp)+'_Mc'+str(Mc)+'_eta'+str(eta) 
               
                
    w_vals = np.arange(pam.wmin, pam.wmax, pam.eta)
    Aw = np.zeros(len(w_vals))
    Aw_dd_total = np.zeros(len(w_vals))
    Aw_d8_total = np.zeros(len(w_vals))

    # set up H0
    if Norb==8 or Norb==5:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,tpd,tpp,0,0,0,0)
    elif Norb==10 or Norb==12:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,0,0,pds,pdp,pps,ppp)
        
    if Norb==5:  
        tapzd_nn_hop_dir, tapzd_orbs, tapzd_nn_hop_fac\
                                   = ham.set_tapzd(Norb,tapzd)        
        
    
    tz_fac = ham.set_tz(Norb,if_tz_exist,tz_a1a1,tz_b1b1)
            
    T_pd   = ham.create_tpd_nn_matrix(VS,tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac)
    T_pp   = ham.create_tpp_nn_matrix(VS,tpp_nn_hop_fac)  
    T_z    = ham.create_tz_matrix(VS,tz_fac)
    T_apzd   = ham.create_tapzd_nn_matrix(VS,tapzd_nn_hop_dir, tapzd_orbs, tapzd_nn_hop_fac)
    Esite  = ham.create_edep_diag_matrix(VS,ANi,ACu,epNi,epCu,epbilayer)      
    
    H0 = T_pd + T_pp + T_z + T_apzd + Esite    
#     H0 = T_pd   
    print("H0 %s seconds ---" % (time.time() - start_time))   
            
    '''
    Below probably not necessary to do the rotation by multiplying U and U_d
    the basis_change.py is only for label the state as singlet or triplet
    and assign the interaction matrix
    '''
    if pam.if_H0_rotate_byU==1:
        H0_Ni_new = U_Ni_d.dot(H0.dot(U_Ni))  
    

    if Norb==5 or Norb==6 or Norb==10 or Norb==11 or Norb==12:     
        Hint_Ni = ham.create_interaction_matrix_ALL_syms(VS,d_Ni_double,p_double,double_Ni_part, idx_Ni, hole345_Ni_part,  \
                                                      S_Ni_val, Sz_Ni_val,AorB_Ni_sym, ACu, ANi, Upp)
        Hint_Cu = ham.create_interaction_matrix_ALL_syms(VS,d_Cu_double,p_double,double_Cu_part, idx_Cu, hole345_Cu_part, \
                                                      S_Cu_val, Sz_Cu_val,AorB_Cu_sym, ACu, ANi, Upp)        
        
        if pam.if_H0_rotate_byU==1:
            H_Ni = H0_Ni_new + Hint_Ni
            
            # continue rotate the basis for setting Cu layer's interaction (d_Cu_double)
            H0_Cu_new = U_Cu_d.dot(H_Ni.dot(U_Cu)) 
            H = H0_Cu_new + Hint_Cu
        else:
            H = H0 + Hint_Ni + Hint_Cu

        print("interaction %s seconds ---" % (time.time() - start_time))               
#         if pam.basis_change_type=='all_states':
#             U_other, S_Ni_val, Sz_Ni_val, S_Cu_val, Sz_Cu_val \
#                 = basis.create_singlet_triplet_basis_change_matrix_other_states(VS, d_Ni_double, d_Cu_double)
#             U_other_d = (U_other.conjugate()).transpose()  
            
#             H = U_other_d.dot(H.dot(U_other))
        
        H.tocsr()

        ####################################################################################
        # compute GS only for turning on full interactions
        if pam.if_get_ground_state==1:
            vals, vecs = gs.get_ground_state(H, VS, S_Ni_val,Sz_Ni_val,S_Cu_val,Sz_Cu_val)
        print("ground state %s seconds ---" % (time.time() - start_time))                   
                
#             if Norb==8:
#                 util.write_GS('Egs_'+flowpeak+'.txt',A,ep,tpd,vals[0])
#             elif Norb==10 or Norb==11 or Norb==12:
#                 util.write_GS2('Egs_'+flowpeak+'.txt',A,ep,pds,pdp,vals[0])
        #########################################################################
        '''
        Compute A(w) for various states
        '''
        if pam.if_compute_Aw==1:
            clf()
#             # compute d8
#             fig.compute_Aw_d8_sym(H, VS, d_double, S_val, Sz_val, AorB_sym, A, w_vals, "Aw_d8_sym_", fname)

            #compute d9Ld9L
            d9Ld9L_a1L_b1L_state_indices, d9Ld9L_a1L_b1L_state_labels, \
                    = getstate.get_d9Ld9L_state_indices(VS)
            fig.compute_Aw1(H, VS, w_vals,  d9Ld9L_a1L_b1L_state_indices, d9Ld9L_a1L_b1L_state_labels, "Aw_d9Ld9L_a1L_b1L__", fname)
           
        
#             #compute d9d9L2        
#             d9d9L2_a1_b1L2_state_indices, d9d9L2_a1_b1L2_state_labels, \
#                     = getstate.get_d9d9L2_state_indices(VS)
#             fig.compute_Aw1(H, VS, w_vals,  d9d9L2_a1_b1L2_state_indices, d9d9L2_a1_b1L2_state_labels, "Aw_d9d9L2_a1_b1L2__", fname)
#             #compute d9L2d9        
#             d9L2d9_a1L2_b1_state_indices, d9L2d9_a1L2_b1_state_labels, \
#                     = getstate.get_d9L2d9_state_indices(VS)
#             fig.compute_Aw1(H, VS, w_vals,  d9L2d9_a1L2_b1_state_indices, d9L2d9_a1L2_b1_state_labels, "Aw_d9L2d9_a1L2_b1__", fname)
                    
#             fig.compute_Aw_d8d8_sym(H, VS, d_Ni_double, S_Ni_val, Sz_Ni_val, AorB_Ni_sym, ANi, S_Cu_val, \
#                                     Sz_Cu_val,AorB_Cu_sym, ACu, w_vals, "Aw_d8d8_sym_", fname)    
##########################################################################
if __name__ == '__main__': 
    Mc  = pam.Mc
    print ('Mc=',Mc)

    Norb = pam.Norb
    eta  = pam.eta
    edNi = pam.edNi
    edCu = pam.edCu

    ANis = pam.ANis
    ACus = pam.ACus
    B  = pam.B
    C  = pam.C
    
    tz_a1a1 = pam.tz_a1a1
    tz_b1b1 = pam.tz_b1b1    
    
    if_tz_exist  = pam.if_tz_exist    
    
    # set up VS
    VS = vs.VariationalSpace(Mc)
#     basis.count_VS(VS)
    print("vs %s seconds ---" % (time.time() - start_time))      
    
    d_Ni_double, idx_Ni, hole345_Ni_part,  double_Ni_part, \
    d_Cu_double, idx_Cu, hole345_Cu_part,  double_Cu_part, \
    p_double,pz_double = ham.get_double_occu_list(VS)
    
    # change the basis for d_double states to be singlet/triplet
    
    if pam.basis_change_type =='all_states':
        U_Ni,S_Ni_val, Sz_Ni_val, AorB_Ni_sym,\
                                        =  basis.create_singlet_triplet_basis_change_matrix \
                                        (VS, double_Ni_part, idx_Ni, hole345_Ni_part,d_Ni_double, d_Cu_double, 'Ni')
        U_Cu,S_Cu_val, Sz_Cu_val, AorB_Cu_sym,\
                                        =  basis.create_singlet_triplet_basis_change_matrix \
                                        (VS, double_Cu_part, idx_Cu, hole345_Cu_part,d_Ni_double, d_Cu_double, 'Cu')

    if pam.basis_change_type =='d_double':
        U_Ni,S_Ni_val, Sz_Ni_val, AorB_Ni_sym,\
                     =  basis.create_singlet_triplet_basis_change_matrix_d_double(VS, d_Ni_double, double_Ni_part, idx_Ni, hole345_Ni_part)
        U_Cu,S_Cu_val, Sz_Cu_val, AorB_Cu_sym,\
                     =  basis.create_singlet_triplet_basis_change_matrix_d_double(VS, d_Cu_double, double_Cu_part, idx_Cu, hole345_Cu_part)    
    print("basis %s seconds ---" % (time.time() - start_time))    
        
    if pam.if_print_VS_after_basis_change==1:
        basis.print_VS_after_basis_change(VS,S_val,Sz_val)
            
#         U = U_Ni+U_Cu
#         print(U_Ni)
#         print(U_Cu)    
    U_Ni_d = (U_Ni.conjugate()).transpose()
    U_Cu_d = (U_Cu.conjugate()).transpose()    
    
    
    # check if U if unitary
#     util.checkU_unitary(U_Ni,U_Ni_d)
    
    if Norb==8 or Norb==5:
            for tpd in pam.tpds:
                for tapzd in pam.tapzds:                
                    for epCu in pam.epCus:
                        for epNi in pam.epNis: 
                            for epbilayer in pam.epbilayers:                         
                                for ANi in pam.ANis:
                                    for ACu in pam.ACus:
            #                            util.get_atomic_d8_energy(ANi,B,C)
                                        for tpp in pam.tpps:
                                            for Upp in pam.Upps:
                                                print ('===================================================')
                                                print ('ANi=',ANi, 'ACu=',ACu,'epCu=', epCu, 'epNi=',epNi,\
                                                       ' tpd=',tpd,' tpp=',tpp,' Upp=',Upp ,'tz_a1a1=',tz_a1a1,'tz_b1b1=',tz_b1b1)

                                                compute_Aw_main(ANi,ACu,epCu,epNi,epbilayer,tpd,tpp,tapzd,tz_a1a1,tz_b1b1,0,0,0,0,Upp,\
                                                                d_Ni_double,d_Cu_double,p_double,pz_double,double_Ni_part,hole345_Ni_part,\
                                                                double_Cu_part,hole345_Cu_part, idx_Ni,idx_Cu, \
                                                                U_Ni, S_Ni_val, Sz_Ni_val, AorB_Ni_sym ,U_Cu, \
                                                                S_Cu_val, Sz_Cu_val, AorB_Cu_sym)  
    elif Norb==9 or Norb==10 or Norb==11:
        pps = pam.pps
        ppp = pam.ppp
        for a in range(0,len(pam.tzs)):
            tz = pam.tzs[a]                                        
            for ii in range(0,len(pam.pdps)):
                pds = pam.pdss[ii]
                pdp = pam.pdps[ii]
                for epCu in pam.epCus:
                    for epNi in pam.epNis:
                        for ANi in pam.ANis: 
                            for ACu in pam.ACus:
    #                         util.get_atomic_d8_energy(ANi,B,C)
                                for Upp in pam.Upps:
                                    print ('===================================================')
                                    print ('ANi=',ANi, 'ACu=',ACu,'epCu=',epCu, 'epNi=',epNi,' pds=',pds,\
                                           ' pdp=',pdp,' pps=',pps,' ppp=',ppp,' Upp=',Upp,'tz=',tz)
                                    compute_Aw_main(ANi,ACu,epCu,epNi,0,0,tz,pds,pdp,pps,ppp,Upp,\
                                                   d_Ni_double,d_Cu_double,p_double,double_Ni_part,hole34_Ni_part,\
                                                   double_Cu_part,hole34_Cu_part,idx_Ni,idx_Cu,\
                                                   U_Ni, S_Ni_val, Sz_Ni_val, AorB_Ni_sym ,U_Cu, \
                                                    S_Cu_val, Sz_Cu_val, AorB_Cu_sym) 

                        
    print("--- %s seconds ---" % (time.time() - start_time))

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Ni_Cu_orbs =  ['d3z2r2', 'dx2y2']
O1_orbs =  ['px']
O2_orbs =  ['py']
Obilayer_orbs =  ['apz']
compute A(w) for symmetries =  ['1A1', '3B1', '3B1', '1A2', '3A2', '1E', '3E']
Mc= 2
VS.dim =  43164
vs 206.64213609695435 seconds ---
len(d_Ni_list) 6360
len(d_Cu_list) 6360
len(p_list) 12608
len(apz_list) 1576
len(idx_Ni) 6360
len(idx_Cu) 6360
basis 2.133833408355713 seconds ---
basis 2.1186437606811523 seconds ---
basis 211.44111728668213 seconds ---
ANi= 6.0 ACu= 6.0 epCu= 2.89 epNi= 2.89  tpd= 1.3  tpp= 0.55  Upp= 0 tz_a1a1= 0.028 tz_b1b1= 0.047
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_tz_matrix
start create_tapzd_nn_matrix
start create_edep_diag_matrix
H0 245.58688640594482 seconds ---
start create_interaction_matrix
--- create_interaction_matrix_ALL_syms 1.1628906726837158 seconds ---
start create_interaction_matrix
---

 state  5934   d3z2r2 up 0 0 2   dx2y2 up 0 0 0   py up 0 -1 2   py dn 0 1 2   px dn -1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008143965974014209 

 state  24652   d3z2r2 up 0 0 2   dx2y2 up 0 0 0   py up 0 -1 2   py dn 0 1 2   py dn 0 1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008143965974014209 

 state  6193   d3z2r2 up 0 0 2   dx2y2 up 0 0 0   px dn -1 0 2   px up 1 0 2   px dn -1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008143965974014209 

 state  35656   d3z2r2 up 0 0 2   dx2y2 up 0 0 0   py up 0 1 2   py dn 0 1 2   px dn 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008143965974014209 

 state  30500   dx2y2 up 0 0 2   d3z2r2 up 0 0 0   py dn 0 -1 2   px dn -1 0 0   px up 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008143965974014208 

 state  21103   dx2y2 up 0 0 2   d3z2r2 up 0 0 0   py dn 0 -1 2   py up 0 -1 0   py dn 0 1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu=

 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008136834034955672 

 state  30927   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   py up 0 1 2   px up -1 0 0   px dn 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008136834034955669 

 state  21102   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   py up 0 -1 2   py up 0 -1 0   py dn 0 1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008136834034955669 

 state  31108   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   px up 1 0 2   px dn -1 0 0   px up 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008136834034955667 

 state  4633   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   py up 0 1 2   px up -1 0 0   px dn -1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008136834034955664 

 state  7975   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   px up 1 0 2   py up 0 -1 0   py dn 0 -1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008136834034955662 

 state  41838   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   px 

 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008011092205475239 

 state  9117   d3z2r2 up 0 0 2   dx2y2 dn 0 0 0   px dn -1 0 2   py up 0 1 2   py up 0 -1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008011092205475239 

 state  35949   d3z2r2 up 0 0 2   dx2y2 dn 0 0 0   py dn 0 -1 2   px up 1 0 2   px up 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008011092205475236 

 state  31886   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   py up 0 -1 2   py up 0 -1 0   px dn 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008011092205475234 

 state  9754   d3z2r2 up 0 0 2   dx2y2 dn 0 0 0   py up 0 1 2   px dn 1 0 2   py up 0 -1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008011092205475234 

 state  25498   d3z2r2 up 0 0 2   dx2y2 dn 0 0 0   py up 0 1 2   px dn 1 0 2   py up 0 1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0008011092205475234 

 state  5848   d3z2r2 up 0 0 2   dx2y2 dn 0 0 0   px up 

 state  23216   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   dx2y2 dn 0 0 0   py dn 0 -1 2   py up 0 1 0 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0007063212892075734 

 state  35562   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   dx2y2 dn 0 0 0   py dn 0 1 2   px up 1 0 0 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0007063212892075732 

 state  34000   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   dx2y2 dn 0 0 0   py dn 0 -1 2   px up 1 0 0 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.000706321289207573 

 state  33704   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   dx2y2 dn 0 0 0   px dn -1 0 2   px up 1 0 0 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.000706321289207573 

 state  5182   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   dx2y2 dn 0 0 0   py dn 0 -1 2   px up -1 0 0 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0007063212892075728 

 state  8448   d3z2r2 up 0 0 2   dx2y2 up 0 0 2   dx2y2 dn 0 0 0   py dn 0 -1 2   py up 0 -1 0 
 S_Ni= 1 ,  Sz_Ni= 1 ,  S_C

...

 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0001093911091788563 

 state  4760   d3z2r2 up 0 0 2   dx2y2 up 0 0 0   px up -1 0 2   px dn -1 0 2   px dn -1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0001093911091788563 

 state  9203   d3z2r2 up 0 0 2   dx2y2 up 0 0 0   py dn 0 -1 2   py up 0 1 2   py dn 0 -1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.00010939110917885627 

 state  5937   d3z2r2 up 0 0 2   dx2y2 up 0 0 0   py dn 0 -1 2   py up 0 1 2   px dn -1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.00010939110917885627 

 state  35436   d3z2r2 up 0 0 2   dx2y2 up 0 0 0   py up 0 -1 2   py dn 0 1 2   px dn 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.00010939110917885623 

 state  9456   d3z2r2 up 0 0 2   dx2y2 up 0 0 0   px up -1 0 2   px dn 1 0 2   py dn 0 -1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.00010939110917885623 

 state  9840   d3z2r2 up 0 0 2   dx2y2 up 0 0 0   

 state  21548   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   py up 0 1 2   py dn 0 -1 0   py up 0 1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.00010929531178398535 

 state  41960   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   py up 0 -1 2   px up 1 0 0   px dn 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.00010929531178398535 

 state  41838   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   px up -1 0 2   px up 1 0 0   px dn 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.00010929531178398528 

 state  30490   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   py up 0 -1 2   px up -1 0 0   px dn 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.00010929531178398528 

 state  4500   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   py up 0 -1 2   px up -1 0 0   px dn -1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0001092953117839852 

 state  7975   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   px up 1 0 2   py up 0 -1 0   py dn 0 -1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  S

 state  6899   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   py up 0 -1 2   px dn -1 0 0   py up 0 -1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.00010760632655971831 

 state  40077   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   px up 1 0 2   py up 0 1 0   px dn 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0001076063265597182 

 state  31895   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   py up 0 -1 2   py dn 0 -1 0   px up 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0001076063265597182 

 state  32495   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   px up 1 0 2   py up 0 -1 0   px dn 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.0001076063265597182 

 state  7508   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   px up 1 0 2   px dn -1 0 0   py up 0 -1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_Cu= 0 , weight =  0.00010760632655971814 

 state  6776   dx2y2 dn 0 0 2   d3z2r2 up 0 0 0   px up -1 0 2   px up -1 0 0   py dn 0 -1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 0 ,  Sz_

 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0 , weight =  0.0001046338439722286 

 state  24376   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   px up 1 0 2   px dn 1 0 2   py dn 0 1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0 , weight =  0.0001046338439722285 

 state  24009   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   px up -1 0 2   px dn 1 0 2   py dn 0 1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0 , weight =  0.00010463384397222842 

 state  24010   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   px dn -1 0 2   px up 1 0 2   py dn 0 1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0 , weight =  0.00010463384397222842 

 state  11715   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   px dn -1 0 2   px up 1 0 2   py dn 0 -1 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0 , weight =  0.00010463384397222842 

 state  34794   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   px dn -1 0 2   px up 1 0 2   px dn 1 0 0 
 S_Ni= 0 ,  Sz_Ni= 0 ,  S_Cu= 1 ,  Sz_Cu= 0 , weight =  0.00010463384397222839 

 state  10737   d3z2r2 up 0 0 0   dx2y2 dn 0 0 0   

sumweight= 1.0000000000000178
LmLn= 0.0662561290744564
d8Ld8= 0.0053178957671817404
d8d8L= 0.005317895767181802
d9Ld8L= 0.05127649602069082
d8Ld9L= 0.05127649602069022
d9d8L2= 2.7267578892882244e-09
d9L2d8= 0.0194547791411253
d8d9L2= 0.019454779141125478
d9L3d9= 3.532586948980018e-09
d9L2d9L= 0.18723499586728734
d9Ld9L2= 0.18723499586728887
d9d9L3= 3.5325869489774285e-09
d8L3d10= 1.826532215857403e-14
d8L2d10L= 2.6729787855041764e-10
d8Ld10L2= 0.030582322084611476
d8d10L3= 0.0058062821732869346
d9d10L4= 8.257520932555904e-11
d9Ld10L3= 0.05579210996799843
d9L2d10L2= 0.1114070474558882
d9L3d10L= 3.7545728894791543e-10
d9L4d10= 1.846606457137998e-15
s= 1.2722836616542666e-31
ds= 4.5685767092791325e-31
dds= 4.598929689251311e-31
ddds= 8.234583309876726e-32
dddds= 6.326139390902119e-33
0.005590426160984641 0.045630294380841964 0.007151836954750778 0.03847845742609122 0.00559042616098459 0.045630294380841464 0.0071518369547507185 0.03847845742609072
sumweight2= 0.7964122348660954
--- get_gro

In [2]:
data=np.array([1,4])
row=np.array([1,2])
col=np.array([0,1])
a=(sps.coo_matrix((data,(row,col)),shape=(3,3))/2).toarray()
b=(a.conjugate()).transpose()

print(a)
print(b)
c = a.dot(b)
print(c)

[[0.  0.  0. ]
 [0.5 0.  0. ]
 [0.  2.  0. ]]
[[0.  0.5 0. ]
 [0.  0.  2. ]
 [0.  0.  0. ]]
[[0.   0.   0.  ]
 [0.   0.25 0.  ]
 [0.   0.   4.  ]]


In [3]:
a = ["a","b","c","d","e"]
b= [1,3,2,4]
print(a.index("d"))  #数据的查找

3


In [4]:
list[2]

list[2]

In [5]:

Ni_layer.append(3)
print (Ni_layer)

NameError: name 'Ni_layer' is not defined

In [ ]:
if Ni_layer==[1,2,3,3]:
    print (1)

In [ ]:
lst1 = [1, 11, 111]
lst2 = [2, 22, 222]
lst3 = [3, 33, 333]
lst4 = [4, 44, 444]
lst5 = [5, 55, 555]
lst6 = [6, 66, 666]


def lamlist1(l1, l2):
    '''
    reduce the 'for' circulation
    '''     
    funs = []
    for i in l1: 
        for j in l2: 
            x = lambda i=i, j=j: (i,j)
            funs.append(x)

    return funs    
lamlist1(range(-2,3),[0,1])
